In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np

In [5]:
# Load the dataset
dataset = pd.read_csv('./data/reddit_preprocessing.csv')

# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = dataset.dropna()

# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

In [6]:
# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

In [7]:
# Tokenize the sentences for Word2Vec training
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized = [sentence.split() for sentence in X_test_cleaned]


In [9]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=300, window=5, min_count=1, workers=4, sg=1) #sg=1 for Skip-Gram model

In [12]:
# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0)) # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size)) # If no words match, use a zero vector
    return np.array(vectorized_comments)

In [13]:
# Vectorized train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

In [14]:
X_train_word2vec.shape

(29329, 300)

In [15]:
# Define the LightGBM model with your best settings
best_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric='multi_logloss',
    is_unbalance=True,
    class_weight="balanced",
    reg_alpha=0.1,
    reg_lambda=0.1,
    learning_rate= 0.07847695073376533,
    n_estimators=426,
    max_depth=20
)

In [16]:
# Train the LightGBM model on Word2Vec embeddings
best_model.fit(X_train_word2vec, y_train_cleaned)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021368 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 300
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


LGBMClassifier(class_weight='balanced', is_unbalance=True,
               learning_rate=0.07847695073376533, max_depth=20,
               metric='multi_logloss', n_estimators=426, num_class=3,
               objective='multiclass', reg_alpha=0.1, reg_lambda=0.1)

In [17]:
# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)

In [18]:
print(classification_report(y_test_cleaned, y_pred))

              precision    recall  f1-score   support

          -1       0.48      0.48      0.48      1647
           0       0.73      0.73      0.73      2510
           1       0.68      0.69      0.69      3176

    accuracy                           0.65      7333
   macro avg       0.63      0.63      0.63      7333
weighted avg       0.65      0.65      0.65      7333



In [19]:
!pip install optuna

  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np
import optuna
from sklearn.model_selection import cross_val_score

# Load the dataset
dataset = pd.read_csv('./data/reddit_preprocessing.csv')

# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = dataset.dropna()

# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Tokenize the sentences for Word2Vec training
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized = [sentence.split() for sentence in X_test_cleaned]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=5, min_count=1, workers=4, sg=1)  # sg=1 for Skip-Gram model

# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0))  # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))  # If no words match, use a zero vector
    return np.array(vectorized_comments)

# Vectorize train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

# Encode the target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_cleaned)
y_test_encoded = label_encoder.transform(y_test_cleaned)

# Objective function for Optuna hyperparameter tuning
def objective(trial):
    # Suggest hyperparameters to be tuned
    params = {
        "objective": "multiclass",
        "num_class": 3,
        "metric": "multi_logloss",
        "is_unbalance": True,
        "class_weight": "balanced",
        "reg_alpha": 0.1,  # L1 regularization
        "reg_lambda": 0.1,  # L2 regularization
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 20)
    }

    # Initialize the LightGBM model with suggested parameters
    model = LGBMClassifier(**params)

    # Perform cross-validation to evaluate the model performance
    scores = cross_val_score(model, X_train_word2vec, y_train_encoded, cv=3, scoring='accuracy')

    # Return the mean accuracy score across folds
    return scores.mean()

# Create an Optuna study to optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Run 30 trials

# Print the best trial
print("Best trial:")
best_params = study.best_trial.params
print(best_params)

# Train the LightGBM model with the best parameters
best_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance=True,
    class_weight="balanced",
    reg_alpha=0.1,
    reg_lambda=0.1,
    **best_params
)

best_model.fit(X_train_word2vec, y_train_encoded)

# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)

[I 2025-04-14 14:28:42,183] A new study created in memory with name: no-name-418228cb-dedf-4776-9d6a-158c64f42b92


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004315 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19552, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004474 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004990

[I 2025-04-14 14:28:59,638] Trial 0 finished with value: 0.652766928222214 and parameters: {'n_estimators': 714, 'learning_rate': 0.2659848022885048, 'max_depth': 19}. Best is trial 0 with value: 0.652766928222214.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004675 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19552, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

[I 2025-04-14 14:29:07,445] Trial 1 finished with value: 0.6404583777150255 and parameters: {'n_estimators': 834, 'learning_rate': 0.2553868419027257, 'max_depth': 3}. Best is trial 0 with value: 0.652766928222214.


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004311 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19552, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

[I 2025-04-14 14:36:20,606] Trial 28 finished with value: 0.6510963198642327 and parameters: {'n_estimators': 940, 'learning_rate': 0.27635050899214647, 'max_depth': 13}. Best is trial 26 with value: 0.6553242427519389.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005087 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19552, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004512 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004543

[I 2025-04-14 14:36:39,020] Trial 29 finished with value: 0.6540284689269008 and parameters: {'n_estimators': 750, 'learning_rate': 0.22939211367547158, 'max_depth': 17}. Best is trial 26 with value: 0.6553242427519389.


Best trial:
{'n_estimators': 954, 'learning_rate': 0.24265953168244114, 'max_depth': 12}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006464 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


In [22]:
# Print classification report
print(classification_report(y_test_encoded, y_pred))

              precision    recall  f1-score   support

           0       0.53      0.41      0.46      1647
           1       0.73      0.72      0.72      2510
           2       0.66      0.75      0.70      3176

    accuracy                           0.66      7333
   macro avg       0.64      0.62      0.63      7333
weighted avg       0.65      0.66      0.65      7333

